In [98]:
import pandas as pd
import numpy as np
import datetime

# Establecer una semilla para reproducibilidad
np.random.seed(42)

# Crear una lista de categorías
categorias = ['A', 'B', 'C', 'D', 'E']

# Crear una lista de fechas aleatorias
fechas = [datetime.datetime(2023, np.random.randint(1, 13), np.random.randint(1, 29),
                            np.random.randint(0, 24), np.random.randint(0, 60), np.random.randint(0, 60))
          if np.random.random() > 0.2 else None
          for _ in range(1000)]

# Crear un DataFrame
df = pd.DataFrame({'CATEGORÍA': [np.random.choice(categorias) for _ in range(1000)],
                   'FECHA': fechas})

# Introducir valores faltantes en categorías
n_valores_faltantes = 100  # Número de valores faltantes a introducir
indices_a_faltar = np.random.choice(len(df), n_valores_faltantes, replace=False)  # Obtener índices aleatorios
df.loc[indices_a_faltar, 'CATEGORÍA'] = None  # Asignar valores faltantes

df['FECHA'] = df['FECHA'].astype('object')

# Calcular el número de duplicados que deseas agregar (5% de los datos)
porcentaje_duplicados = 0.05
num_duplicados = int(len(df) * porcentaje_duplicados)

# Seleccionar aleatoriamente filas para duplicar
filas_a_duplicar = df.sample(n=num_duplicados, replace=True)

# Agregar las filas duplicadas al DataFrame
df = pd.concat([df, filas_a_duplicar], ignore_index=True)

# Asegurarse de que el DataFrame aún tenga 1000 filas (original + duplicados)
df = df.iloc[:1000]

# Opcionalmente, puedes reordenar las filas si deseas mezclar los duplicados con los originales
df = df.sample(frac=1).reset_index(drop=True)
df

,CATEGORÍA,FECHA
0,C,2023-11-04 12:31:06
1,B,2023-08-15 08:03:13
2,E,NaT
3,E,2023-11-02 17:28:56
4,A,2023-11-05 19:10:09
...,...,...
995,A,2023-01-27 06:05:14
996,A,2023-03-12 08:59:06
997,B,2023-11-22 23:03:27
998,None,2023-08-19 04:57:03


In [97]:
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)

(1000, 2)
(820, 2)


In [60]:
df = df.rename(columns=lambda col: str(col).lower().strip())
df

,categoría,fecha
0,E,2023-01-20 23:09:47
1,C,2023-06-05 20:55:08
2,C,2023-07-13 15:38:17
3,C,NaT
4,C,2023-12-02 20:46:40
...,...,...
995,E,2023-09-06 09:18:53
996,E,NaT
997,A,NaT
998,C,2023-10-18 01:07:06


In [61]:
t = df.filter(regex='fecha|date|tiempo|time').columns
df[t] = df[t].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
df

,categoría,fecha
0,E,2023-01-20 23:09:47
1,C,2023-06-05 20:55:08
2,C,2023-07-13 15:38:17
3,C,NaT
4,C,2023-12-02 20:46:40
...,...,...
995,E,2023-09-06 09:18:53
996,E,NaT
997,A,NaT
998,C,2023-10-18 01:07:06


In [63]:
categoricals = df.select_dtypes(include=['object', 'bool']).columns

df[categoricals] = df[categoricals].applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
df[categoricals]

,categoría
0,e
1,c
2,c
3,c
4,c
...,...
995,e
996,e
997,a
998,c


In [64]:
df.isnull().mean()

categoría    0.100
fecha        0.196
dtype: float64

In [ ]:
# Manejar el nombre de los predictores
df = df.rename(columns=lambda col: str(col).lower().strip())

In [101]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin


class ColumnNameTransformer(BaseEstimator, TransformerMixin):
    
    """
    A transformer for renaming columns of a DataFrame using a custom transformation function.

    Parameters:
    -----------
    transformation: function
        A function that takes a column name (string) as input and returns the new column name.

    Attributes:
    -----------
    transformation: function
        The transformation function used for renaming column names.

    Methods:
    --------
    fit(X, y=None):
        Fit the transformer to the data. Since this transformer doesn't require any training,
        it returns itself unchanged.

    transform(X):
        Rename columns of the input DataFrame X using the provided transformation function.

    Examples:
    ---------
    >>> import pandas as pd
    >>> df = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
    >>> def custom_transform(col_name):
    ...     return col_name.lower()
    >>> transformer = ColumnNameTransformer(transformation=custom_transform)
    >>> df_transformed = transformer.transform(df)
    >>> df_transformed
       a  b
    0  1  3
    1  2  4
    """

    def __init__(self, transformation):
        
        """
        Initialize the transformer with a custom column name transformation function.

        Parameters:
        ----------
        transformation : function
            A function that takes a column name (string) as input and returns the new column name.
        """
        self.transformation = transformation

    def fit(self, X: pd.DataFrame, y=None):
        
        """
        Fit the transformer to the data. Since this transformer doesn't require any training,
        it returns itself unchanged.

        Parameters:
        -----------
        X: pandas.DataFrame
            The input DataFrame.

        y: None
            Ignored. This parameter is included for compatibility with scikit-learn's transformers.

        Returns:
        --------
        self : ColumnNameTransformer
            The fitted transformer instance.
        """
        return self

    def transform(self, X: pd.DataFrame):
        
        """
        Rename columns of the input DataFrame X using the provided transformation function.

        Parameters:
        -----------
        X: pandas.DataFrame
            The input DataFrame with columns to be renamed.

        Returns:
        --------
        X_transformed: pandas.DataFrame
            The DataFrame with column names transformed according to the provided function.
        """
        
        X_transformed = X.rename(columns=self.transformation)
        return X_transformed

    
class DropDuplicatedTransformer(BaseEstimator, TransformerMixin):
    
    """
    A transformer to remove duplicate rows from a DataFrame.

    Parameters:
    -----------
    None

    Attributes:
    -----------
    None

    Methods:
    --------
    fit(X, y=None):
        Fit the transformer to the data. Since this transformer doesn't require any training,
        it returns itself unchanged.

    transform(X):
        Remove duplicate rows from the input DataFrame X.

    Examples:
    ---------
    >>> from sklearn.datasets import load_iris
    >>> iris = load_iris(as_frame=True)
    >>> df = iris.data
    >>> transformer = DropDuplicatedTransformer()
    >>> df_no_duplicates = transformer.transform(df)
    """

    def __init__(self):
        
        """
        Initialize the transformer.

        Parameters:
        -----------
        None
        """
        pass

    def fit(self, X: pd.DataFrame, y=None):
        
        """
        Fit the transformer to the data. Since this transformer doesn't require any training,
        it returns itself unchanged.

        Parameters:
        -----------
        X: pandas.DataFrame
            The input DataFrame.

        y: None
            Ignored. This parameter is included for compatibility with scikit-learn's transformers.

        Returns:
        --------
        self : DropDuplicatedTransformer
            The fitted transformer instance.
        """
        
        return self

    def transform(self, X: pd.DataFrame):
        
        """
        Remove duplicate rows from the input DataFrame X.

        Parameters:
        -----------
        X: pandas.DataFrame
            The input DataFrame from which duplicate rows will be removed.

        Returns:
        --------
        X_no_duplicates: pandas.DataFrame
            The DataFrame with duplicate rows removed.
        """
        
        X_no_duplicates = X.drop_duplicates()
        return X_no_duplicates


In [84]:
# Crear una instancia de la clase ColumnNameTransformer con la transformación deseada
df_transformer = ColumnNameTransformer(lambda col: str(col).lower().strip())

# Aplicar la transformación a tu DataFrame df
df = df_transformer.transform(df)

In [100]:
print(df.shape)

# Crear una instancia de la clase ColumnNameTransformer con la transformación deseada
df_transformer = DropDuplicatedTransformer()

# Aplicar la transformación a tu DataFrame df
df = df_transformer.transform(df)

print(df.shape)

(1000, 2)
(820, 2)


In [110]:
condicion = df.isnull().sum(axis=1) > df.shape[1] / 3
filas_con_nulos = df[condicion]
filas_con_nulos

,CATEGORÍA,FECHA
2,E,NaT
8,None,2023-08-01 11:04:02
12,B,NaT
14,None,NaT
20,C,NaT
...,...,...
969,None,2023-11-23 06:26:28
977,None,2023-04-20 04:53:20
993,None,2023-10-09 07:25:28
994,None,2023-12-22 20:53:10


In [ ]:
df.drop()

In [114]:
df.columns[df.isnull().mean() > 1/3]

Index([], dtype='object')